In [2]:
from my_packages import *
from appgeopy import *
warnings.filterwarnings("ignore")

In [3]:
def export_parameter_record(savefolder, params_dict):

    import os

    for i in range(1, 1000):
        savepath = os.path.join(savefolder, "Param_Record_{}.txt".format(str(i).zfill(3)))
        if os.path.exists(savepath):
            continue
        else:
            break

    with open(savepath, "w+") as file:
        for key, value in params_dict.items():
            file.write("{} : {}".format(key, value))
            file.write("\n\n")

    pop_up_string = "Saved at {}".format(savefolder)
    print(pop_up_string)


def open_geodata(filepath):
    import sys

    extension = filepath.split(".")[-1]

    if extension == "pkl":
        geodata = pd.read_pickle(filepath, compression="zip")
    elif extension == "shp":
        geodata = gpd.read_file(filepath)
    else:
        sys.exit(
            "The datatype of PSC_filepath is invalid.\
        \nExpect {} or {}, not {}".format(
                type("1"), geodata_type, type(PSC_filepath)
            )
        )
    return geodata

#### Data Input

In [7]:
file2process = [r"D:\003_PRESENTATIONS\20_TSX_Sentinel_Report\hsr_section\TSX_HSR_Buffer_200m_TWD97.pkl"]

basefolder = os.path.dirname(profile_segment_psc_folder)

savefolder = r"D:\003_PRESENTATIONS\20_TSX_Sentinel_Report\hsr_section"

if not os.path.exists(savefolder):
    os.makedirs(savefolder)

params_dict = {
    "profile_segment_psc_folder": profile_segment_psc_folder,
    "file2process": file2process,
    "savefolder": savefolder,
}

#### Function Processing

In [8]:
file = file2process[0]
data = open_geodata(file)
data.head(5)

,OBJECTID,CODE,HEIGHT,H_STDEV,VEL,V_STDEV,COHERENCE,EFF_AREA,POINT_X,POINT_Y,PointKey,velocity_mm_per_day,velocity_mm_per_year,Segment,geometry
0,30460,000076fc,42.410000,0.0,9.580000,5.02,0.999,1,191442.0792,2.628111e+06,X191442Y2628111,-0.177695,-64.903113,Seg_28500,POINT (191442.079 2628111.332)
1,28177,00006e11,42.520001,0.0,16.709999,5.85,1.000,1,191065.9593,2.628185e+06,X191065Y2628184,-0.158085,-57.740586,Seg_28550,POINT (191065.959 2628184.677)
2,28178,00006e12,43.020001,0.0,17.820000,4.65,0.998,1,191077.6163,2.628183e+06,X191077Y2628182,-0.140757,-51.411453,Seg_28550,POINT (191077.616 2628182.517)
3,28179,00006e13,43.880001,0.0,18.740000,4.80,0.999,1,191087.7188,2.628181e+06,X191087Y2628180,-0.134751,-49.217728,Seg_28550,POINT (191087.719 2628180.575)
4,28569,00006f99,44.709999,0.0,22.000000,5.00,0.997,1,191098.5981,2.628178e+06,X191098Y2628178,-0.149761,-54.700289,Seg_28550,POINT (191098.598 2628178.419)


In [4]:
data = data.loc[:, "velocity_mm_per_year":"Segment"]

unique_segment = np.unique(data["Segment"].values)

amount_list = []

empty_mean = pd.DataFrame(data=None)
empty_stdev = pd.DataFrame(data=None)

for segment_name in unique_segment:

    data_by_segment = data[data["Segment"] == segment_name]
    average_by_segment = data_by_segment.groupby("Segment").mean()
    stdev_by_segment = data_by_segment.groupby("Segment").std()

    amount_list.append(len(data_by_segment))

    empty_mean = pd.concat([empty_mean, average_by_segment])
    empty_stdev = pd.concat([empty_stdev, stdev_by_segment])

empty_mean["Pts_Num"] = amount_list
empty_stdev["Pts_Num"] = amount_list

# savename_mean = "Mean_" + os.path.basename(file)[:-4] + ".xlsx"
# savename_stdev = "Stdev_" + os.path.basename(file)[:-4] + ".xlsx"

# empty_mean.to_excel(os.path.join(savefolder, savename_mean))
# empty_stdev.to_excel(os.path.join(savefolder, savename_stdev))

Saved at E:\001_InSAR_Project\4000_TERRASAR-X\TSX_Choushui\14_POSTPROCESS\HSR_PROFILE\PSC_CSV_Export
